In [1]:
import getEPH
import functionsForModels
import make_dummy
import schoolYears
import categorize
import functionsForModels
import createVariables

import pandas as pd
#http://statsmodels.sourceforge.net/devel/examples/generated/example_wls.html
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from statsmodels.iolib.table import (SimpleTable, default_txt_fmt)
np.random.seed(1024)
%matplotlib inline

/home/pipe/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
getEPH.getEPHdbf('t310')

In [2]:
data1 = pd.read_csv('data/cleanDatat310.csv')
data2 = categorize.categorize(data1)
data3 = schoolYears.schoolYears(data2)
data4 = createVariables.createVariables(data3)


In [13]:
data4.PONDERA.sum()

13059409

In [4]:
data = data4.copy()

In [7]:
jobsAndIncome = (data.activity == 1) & (data.lnIncome > 1) & ((data.familyRelation == 1)|(data.familyRelation == 2))
cleanData = data.copy().loc[jobsAndIncome,
                              ['id',
                            'familyRelation',
                              'age',
                              'age2',
                              'female',
                              'education',
                              'education2']]




In [8]:
cleanData.head()

,id,familyRelation,age,age2,female,education,education2
2,3078611,1,42,1764,0,7.0,49.0
3,3078611,2,44,1936,1,17.0,289.0
7,3082781,1,38,1444,0,17.0,289.0
8,3082781,2,28,784,1,17.0,289.0
9,3119371,1,63,3969,0,17.0,289.0


In [10]:
pivot = cleanData.pivot(index='id', columns='familyRelation')
pivot.head()

age          age2        female      education        \
familyRelation     1     2       1      2      1    2         1     2   
id                                                                      
1250971         57.0   NaN  3249.0    NaN    1.0  NaN      14.0   NaN   
1256691         61.0   NaN  3721.0    NaN    0.0  NaN       7.0   NaN   
1256892         31.0  23.0   961.0  529.0    0.0  1.0      12.0  12.0   
1261181         38.0  30.0  1444.0  900.0    0.0  1.0       7.0   7.0   
1261521         25.0  25.0   625.0  625.0    1.0  0.0      16.0  12.0   

               education2         
familyRelation          1      2  
id                                
1250971             196.0    NaN  
1256691              49.0    NaN  
1256892             144.0  144.0  
1261181              49.0   49.0  
1261521             256.0  144.0

In [11]:
np.where(np.isnan(pivot.values),0,pivot.values)


array([[   57.,     0.,  3249., ...,     0.,   196.,     0.],
       [   61.,     0.,  3721., ...,     0.,    49.,     0.],
       [   31.,    23.,   961., ...,    12.,   144.,   144.],
       ..., 
       [   43.,     0.,  1849., ...,     0.,   144.,     0.],
       [    0.,    66.,     0., ...,     7.,     0.,    49.],
       [   28.,     0.,   784., ...,     0.,    49.,     0.]])

In [12]:
pivot.shape

(1916, 10)